In [6]:
import os, sys
import torch, json
import numpy as np
import torch.nn as nn
from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops


# 0. Initialize and Load Pre-trained Models


In [2]:
model_config_path = "config/DINO/DINO_4scale.py" # change the path of the model config file
model_checkpoint_path = "ckpt/checkpoint0023_4scale.pth" # change the path of the model checkpoint
# See our Model Zoo section in README.md for more details about our pretrained models.

In [4]:
args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
model.eval();

In [7]:
model_quat = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

### Model size comparison


In [11]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

In [13]:
# compare the sizes
f=print_size_of_model(model,"fp32")
q=print_size_of_model(model_quat,"int8")
print("{0:.2f} times smaller".format(f/q))

model:  fp32  	 Size (KB): 188168.057
model:  int8  	 Size (KB): 142153.754
1.32 times smaller
